## Group 7
## Big Data Group Project

### Contributors:
- 22606127: Sean Whitehead [Sean]
- 29650437: Lin Bai

### Phase 2: Cleansing and Reshaping

This notebook is used for co-developing of codes required to wrangle the home credit risk dataset.

Please use this notebook as the live version and download local copies as you require.

If you have picked up a specific section, please comment your name to the code cell so that we can work on other sections.

Helpful link for Dataframes in Scala:
- https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-scala.html#work-with-dataframes
- https://spark.apache.org/docs/1.5.0/api/java/overview-summary.html

Insert libraries into the cell below:

In [1]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql._
import org.apache.hadoop.io.LongWritable
import org.apache.hadoop.io.Text
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.mapreduce.lib.input.TextInputFormat

Intitializing Scala interpreter ...

Spark Web UI available at http://95a19fa2a889:4041
SparkContext available as 'sc' (version = 2.4.3, master = local[*], app id = local-1558766339609)
SparkSession available as 'spark'


import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql._
import org.apache.hadoop.io.LongWritable
import org.apache.hadoop.io.Text
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.mapreduce.lib.input.TextInputFormat


Insert code for data loading below:

Download zip from GDrive and extract to location where you are running notebook for below to run

In [2]:
//read in csv [Sean]
val creditRiskdf = spark.read.option("header","true").csv("Home credit default risk after feature selection/m_train_small.csv")

creditRiskdf: org.apache.spark.sql.DataFrame = [EXT_SOURCE_1: string, EXT_SOURCE_2: string ... 359 more fields]


In [15]:
//take a look at features in dataset (...361 of them) [Sean]
creditRiskdf.printSchema()

root
 |-- EXT_SOURCE_1: string (nullable = true)
 |-- EXT_SOURCE_2: string (nullable = true)
 |-- EXT_SOURCE_3: string (nullable = true)
 |-- client_installments_AMT_PAYMENT_min_sum: string (nullable = true)
 |-- DAYS_BIRTH: string (nullable = true)
 |-- AMT_CREDIT: string (nullable = true)
 |-- AMT_ANNUITY: string (nullable = true)
 |-- DAYS_EMPLOYED: string (nullable = true)
 |-- bureau_DAYS_CREDIT_ENDDATE_max: string (nullable = true)
 |-- bureau_DAYS_CREDIT_max: string (nullable = true)
 |-- DAYS_ID_PUBLISH: string (nullable = true)
 |-- bureau_AMT_CREDIT_MAX_OVERDUE_mean: string (nullable = true)
 |-- bureau_DAYS_ENDDATE_FACT_max: string (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_DEBT_mean: string (nullable = true)
 |-- OWN_CAR_AGE: string (nullable = true)
 |-- previous_loans_CNT_PAYMENT_mean: string (nullable = true)
 |-- client_cash_CNT_INSTALMENT_FUTURE_min_max: string (nullable = true)
 |-- previous_loans_SELLERPLACE_AREA_max: string (nullable = true)
 |-- DAYS_LAST_PHONE_C

In [3]:
//count how many rows we are working with
creditRiskdf.count()

2019-05-25 06:40:45,701 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


res0: Long = 307511


In [16]:
//string array of features to pass through EDA functions [Sean]
val creditRiskFeatures = creditRiskdf.columns

creditRiskFeatures: Array[String] = Array(EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3, client_installments_AMT_PAYMENT_min_sum, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_EMPLOYED, bureau_DAYS_CREDIT_ENDDATE_max, bureau_DAYS_CREDIT_max, DAYS_ID_PUBLISH, bureau_AMT_CREDIT_MAX_OVERDUE_mean, bureau_DAYS_ENDDATE_FACT_max, bureau_AMT_CREDIT_SUM_DEBT_mean, OWN_CAR_AGE, previous_loans_CNT_PAYMENT_mean, client_cash_CNT_INSTALMENT_FUTURE_min_max, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, bureau_AMT_CREDIT_SUM_max, bureau_AMT_CREDIT_SUM_mean, DAYS_REGISTRATION, client_installments_DAYS_INSTALMENT_max_max, previous_loans_AMT_DOWN_PAYMENT_max, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_UPDATE_max, bureau_AMT_CREDIT_SUM_sum, clie...

Insert code for flagging missing values below:

In [6]:
import org.apache.spark.sql.functions.{sum, col}

import org.apache.spark.sql.functions.{sum, col}


In [7]:
creditRiskdf.select(creditRiskdf.columns.map(c => sum(col(c).isNull.cast("int")).alias(c)): _*).show

+------------+------------+------------+---------------------------------------+----------+----------+-----------+-------------+------------------------------+----------------------+---------------+----------------------------------+----------------------------+-------------------------------+-----------+-------------------------------+-----------------------------------------+-----------------------------------+----------------------+-------------------------+--------------------------+-----------------+-------------------------------------------+-----------------------------------+-------------+--------------------------+-----------------------------------+-----------------------------+-------------------------+-------------------------------------------+---------------------------+---------------------------------------------------+-----------------------+----------------------------------------+-------------------------------------------+--------------------------------------------

In [8]:
val df1=creditRiskdf.na.drop()//Whether we should drop the na is still under discussion
df1.count()

df1: org.apache.spark.sql.DataFrame = [EXT_SOURCE_1: string, EXT_SOURCE_2: string ... 359 more fields]
res4: Long = 865


Insert code for flagging duplicate values below:

In [9]:
val df2=df1.dropDuplicates() //We can see from the count() output that here are no duplicates in this dataframe.
df2.count()

df2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_1: string, EXT_SOURCE_2: string ... 359 more fields]
res5: Long = 865


Insert code for visualising or displaying outliers below:

Insert code for reviewing data values for inconsistencies below:

Insert code for reviewing data format below:

In [ ]:
// pick out column headers containing 'date' [JC]
val matcher = ".+date.+"
val dates = creditRiskFeatures.filter(x => x.toLowerCase matches matcher)

In [ ]:
// review data format of columnbs with headers containing 'date' [JC]

for (dateformat <- dates){
creditRiskdf.select(dateformat).show(10)
}

No data containing date format. [JC]

Insert code for encoding categorical variables below: